#Task 
Predict ว่าคนหนึ่งๆจะเรียนจบคอร์สหนึ่งๆหรือไม่
#Dataset (df2)
  Feature ทุุุกอย่างเหมือนอีกแบบหนึ่งยกเว้น
*   avg_score 
*   quiz_submit
*   quiz_open

ผมไม่ได้เอา record ทั้งหมดมาทำ feature แต่ตัดเอา record ครึ่งนึงมาทำ feature






In [0]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import lightgbm as lgbm
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,ExtraTreesClassifier
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle
from sklearn.metrics import classification_report




In [0]:
#df2=pd.read_csv("dfhalf.csv")
df2=pd.read_csv("dfneweraaisus.csv")
df2=df2.drop(columns=["all_score","cv_uid","cv_cid"])
df2["SSS"]=df2["quiz_submit"]+df2["quiz_open"]
df2 = shuffle(df2)
X_train2=pd.DataFrame(df2)
Y_train2=X_train2["label"]
X_train2=X_train2.drop(columns=["label"])
Layer1DF=pd.DataFrame(df2)


In [52]:
df2.describe()


,label,enroll_count,complete_count,quiz_submit,quiz_open,nub,avg_score,did_watch,G1,G2,G3,SSS
count,109470.000000,109470.000000,109470.000000,109470.000000,109470.000000,109470.000000,109470.000000,109470.000000,109470.000000,109470.000000,109470.000000,109470.000000
mean,0.211446,2.352846,0.563296,4.182571,3.638970,2.140714,0.220349,0.243537,0.303078,0.493213,0.053549,7.821540
std,0.408336,3.537913,1.571846,10.878649,7.219468,3.769802,0.354274,0.429218,0.459591,0.499956,0.225126,17.053703
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,0.000000,3.000000,0.000000,2.000000,3.000000,2.000000,0.500000,0.000000,1.000000,1.000000,0.000000,5.000000
max,1.000000,32.000000,26.000000,358.000000,202.000000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,386.000000


In [60]:
df2[df2["nub"]>15]["nub"].count()

1368

In [61]:
df2[df2["SSS"]>50].count()

label             4303
enroll_count      4303
complete_count    4303
quiz_submit       4303
quiz_open         4303
nub               4303
avg_score         4303
did_watch         4303
G1                4303
G2                4303
G3                4303
SSS               4303
dtype: int64

In [30]:
df2[(df2['avg_score']==0) & (df2['label']==1)].describe()

,label,enroll_count,complete_count,avg_score,did_watch,G1,G2,G3
count,21617.0,21617.000000,21617.000000,21617.0,21617.000000,21617.000000,21617.000000,21617.000000
mean,1.0,2.092612,1.056206,0.0,0.531989,0.277837,0.410233,0.044224
std,0.0,3.365218,2.154049,0.0,0.498987,0.447943,0.491887,0.205598
min,1.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,1.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,1.0,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000
75%,1.0,3.000000,1.000000,0.0,1.000000,1.000000,1.000000,0.000000
max,1.0,30.000000,26.000000,0.0,1.000000,1.000000,1.000000,1.000000



#ลองเอา dataset มา Train ก่อน

In [31]:
lg=LogisticRegression()
lg.fit(X_train2[:85000],Y_train2[:85000])
k=lg.score(X_train2[85000:], Y_train2[85000:])
print(k)
lgy=lg.predict(X_train2[85000:])
tn, fp, fn, tp = confusion_matrix(Y_train2[85000:],lgy).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=2*precision*recall/(precision+recall)
print(precision,recall,f1)
print(classification_report(Y_train2[85000:], lgy))

0.8125051082958725
0.6417641764176417 0.2732847834419318 0.38333333333333336
              precision    recall  f1-score   support

           0       0.83      0.96      0.89     19252
           1       0.64      0.27      0.38      5218

    accuracy                           0.81     24470
   macro avg       0.74      0.62      0.64     24470
weighted avg       0.79      0.81      0.78     24470



In [32]:
def build_nn():
  Kclass=Sequential()
  Kclass.add(Dense(15,activation='tanh',input_dim=7))
  Kclass.add(Dropout(0.2))
  Kclass.add(Dense(1,activation='sigmoid'))
  Kclass.compile(optimizer="adam", loss = 'binary_crossentropy', metrics = ['accuracy'])
  return Kclass
KRlayer1=KerasClassifier(build_fn=build_nn,epochs=100,batch_size=10)
KRlayer1.fit(X_train2[:85000],Y_train2[:85000])


Epoch 1/100
85000/85000 [==============================] - 12s 141us/step - loss: 0.4338 - acc: 0.8056
Epoch 2/100
85000/85000 [==============================] - 12s 137us/step - loss: 0.4204 - acc: 0.8129
Epoch 3/100
85000/85000 [==============================] - 11s 134us/step - loss: 0.4198 - acc: 0.8145
Epoch 4/100
85000/85000 [==============================] - 12s 138us/step - loss: 0.4182 - acc: 0.8150
Epoch 5/100
85000/85000 [==============================] - 11s 133us/step - loss: 0.4178 - acc: 0.8155
Epoch 6/100
85000/85000 [==============================] - 11s 133us/step - loss: 0.4173 - acc: 0.8159
Epoch 7/100
85000/85000 [==============================] - 12s 137us/step - loss: 0.4164 - acc: 0.8168
Epoch 8/100
85000/85000 [==============================] - 11s 133us/step - loss: 0.4160 - acc: 0.8172
Epoch 9/100
85000/85000 [==============================] - 12s 136us/step - loss: 0.4160 - acc: 0.8172
Epoch 10/100
85000/85000 [==============================] - 11s 133us/ste

In [33]:
k=KRlayer1.score(X_train2[85000:], Y_train2[85000:])
print(k)
lgy=KRlayer1.predict(X_train2[85000:])
tn, fp, fn, tp = confusion_matrix(Y_train2[85000:],lgy).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=2*precision*recall/(precision+recall)
print(precision,recall,f1)
print(classification_report(Y_train2[85000:], lgy))

24470/24470 [==============================] - 2s 68us/step
0.8308949692180121
0.7757916241062308 0.2911077041011882 0.42335562987736897
              precision    recall  f1-score   support

           0       0.84      0.98      0.90     19252
           1       0.78      0.29      0.42      5218

    accuracy                           0.83     24470
   macro avg       0.81      0.63      0.66     24470
weighted avg       0.82      0.83      0.80     24470



In [34]:
lg=lgbm.LGBMClassifier()
lg.fit(X_train2[:85000],Y_train2[:85000])
k=lg.score(X_train2[85000:], Y_train2[85000:])
print(k)
lgy=lg.predict(X_train2[85000:])
tn, fp, fn, tp = confusion_matrix(Y_train2[85000:],lgy).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=2*precision*recall/(precision+recall)
print(precision,recall,f1)
print(classification_report(Y_train2[85000:], lgy))
lg.feature_importances_

0.83334695545566
0.7745664739884393 0.3081640475277884 0.4409103372635042
              precision    recall  f1-score   support

           0       0.84      0.98      0.90     19252
           1       0.77      0.31      0.44      5218

    accuracy                           0.83     24470
   macro avg       0.81      0.64      0.67     24470
weighted avg       0.83      0.83      0.80     24470



array([1115,  915,  275,  138,  197,  217,  143])

# ที่ต่างจากแบบแรกคือ LogisticRegression มี f1 score ที่ลดลงมากเหลือประมาณ 60% และทุกโมเดลมี accuracy ลดลง

#Stacking

In [35]:
LRlayer1=LogisticRegression()
LRlayer1.fit(X_train2,Y_train2)
out=LRlayer1.predict_proba(X_train2)
Layer1DF["LRProb"]=out[:,1]
Layer1DF

,label,enroll_count,complete_count,avg_score,did_watch,G1,G2,G3,LRProb
103712,0,0,0,0.0,0,0,1,0,0.117962
81110,0,0,0,0.0,0,0,1,0,0.117962
52914,0,2,0,0.0,0,0,0,0,0.143706
39690,0,0,0,0.0,0,0,0,0,0.194872
83297,0,0,0,0.0,0,0,1,0,0.117962
...,...,...,...,...,...,...,...,...,...
62545,0,1,0,0.0,0,0,0,0,0.167737
51178,0,1,0,0.0,0,0,0,0,0.167737
79051,0,1,1,0.0,0,0,1,0,0.156026
21810,0,3,0,0.0,0,0,1,0,0.071680


In [0]:
lg=lgbm.LGBMClassifier()
lg.fit(X_train2,Y_train2)
out=lg.predict_proba(X_train2)
Layer1DF["LGProb"]=out[:,1]

In [0]:
ada=AdaBoostClassifier()
ada.fit(X_train2,Y_train2)
out=ada.predict_proba(X_train2)
Layer1DF["ADAProb"]=out[:,1]

In [0]:
ext=ExtraTreesClassifier()
ext.fit(X_train2,Y_train2)
out=ext.predict_proba(X_train2)
Layer1DF["EXTProb"]=out[:,1]

In [0]:
kn=KNeighborsClassifier()
kn.fit(X_train2,Y_train2)
out=kn.predict_proba(X_train2)
Layer1DF["KNProb"]=out[:,1]

In [0]:
rf=RandomForestClassifier()
rf.fit(X_train2, Y_train2)
out=rf.predict_proba(X_train2)
Layer1DF["RFProb"]=out[:,1]
#OO=rf.feature_importances_.reshape(1,6)
#for i in range(OO.shape[1]):
#  IM[i]+=OO[0,i]


In [41]:
xgb = XGBClassifier()
xgb.fit(X_train2, Y_train2)
print(xgb.score(X_train2,Y_train2))
out=xgb.predict_proba(X_train2)
Layer1DF["XGProb"]=out[:,1]

0.8309491184799488


In [42]:
def build_nn():
  Kclass=Sequential()
  Kclass.add(Dense(15,activation='tanh',input_dim=7))
  Kclass.add(Dropout(0.2))
  Kclass.add(Dense(1,activation='sigmoid'))
  Kclass.compile(optimizer="adam", loss = 'binary_crossentropy', metrics = ['accuracy'])
  return Kclass
KRlayer1=KerasClassifier(build_fn=build_nn,epochs=100,batch_size=10)
KRlayer1.fit(X_train2,Y_train2)
out=KRlayer1.predict_proba(X_train2)
Layer1DF["KRProb"]=out[:,1]

Epoch 1/100
109470/109470 [==============================] - 15s 141us/step - loss: 0.4271 - acc: 0.8096
Epoch 2/100
109470/109470 [==============================] - 15s 138us/step - loss: 0.4187 - acc: 0.8139
Epoch 3/100
109470/109470 [==============================] - 15s 139us/step - loss: 0.4174 - acc: 0.8149
Epoch 4/100
109470/109470 [==============================] - 15s 134us/step - loss: 0.4162 - acc: 0.8165
Epoch 5/100
109470/109470 [==============================] - 16s 142us/step - loss: 0.4160 - acc: 0.8166
Epoch 6/100
109470/109470 [==============================] - 15s 135us/step - loss: 0.4159 - acc: 0.8172
Epoch 7/100
109470/109470 [==============================] - 15s 136us/step - loss: 0.4151 - acc: 0.8179
Epoch 8/100
109470/109470 [==============================] - 15s 135us/step - loss: 0.4148 - acc: 0.8190
Epoch 9/100
109470/109470 [==============================] - 15s 135us/step - loss: 0.4145 - acc: 0.8200
Epoch 10/100
109470/109470 [===========================

In [0]:
#IM=[[-val,key] for key,val in IM.items()]
#IM.sort()
#print(IM)
#Layer1DF=Layer1DF.drop(columns=["complete_count","did_watch"])
#ayer1DF[Layer1DF["label"]==1]
#Layer1DF

In [0]:
Layer1DF.to_csv("dflayer18.csv",index=False)

In [0]:
Layer1DF=pd.read_csv("dflayer18.csv")

In [46]:
lg=lgbm.LGBMClassifier()
Y=Layer1DF["label"]
MM=Layer1DF.drop(columns=["label"])
lg.fit(MM[:85000],Y[:85000])
k=lg.score(MM[85000:], Y[85000:])
k2=lg.score(MM[:85000], Y[:85000])
print(k2,k)
lgy=lg.predict(MM[85000:])
tn, fp, fn, tp = confusion_matrix(Y[85000:],lgy).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=2*precision*recall/(precision+recall)
print(precision,recall,f1)
print(classification_report(Y_train2[85000:], lgy))

0.834835294117647 0.8343277482631795
0.7662397072278133 0.3210042161747796 0.4524581307401404
              precision    recall  f1-score   support

           0       0.84      0.97      0.90     19252
           1       0.77      0.32      0.45      5218

    accuracy                           0.83     24470
   macro avg       0.80      0.65      0.68     24470
weighted avg       0.83      0.83      0.81     24470



In [47]:
lg=LogisticRegression()
Y=Layer1DF["label"]
MM=Layer1DF.drop(columns=["label"])
lg.fit(MM[:85000],Y[:85000])
k=lg.score(MM[85000:], Y[85000:])
k2=lg.score(MM[:85000], Y[:85000])
print(k2,k)
lgy=lg.predict(MM[85000:])
tn, fp, fn, tp = confusion_matrix(Y[85000:],lgy).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=2*precision*recall/(precision+recall)
print(precision,recall,f1)
print(classification_report(Y_train2[85000:], lgy))

0.834435294117647 0.8356354720065386
0.7743119266055046 0.32349559218091223 0.4563395512300622
              precision    recall  f1-score   support

           0       0.84      0.97      0.90     19252
           1       0.77      0.32      0.46      5218

    accuracy                           0.84     24470
   macro avg       0.81      0.65      0.68     24470
weighted avg       0.83      0.84      0.81     24470



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


# หลังการ stack จะได้ accuracy ประมาณ 95% น้อยกว่าแบบแรก (97%)
